In [24]:
# Relevant module imports and installs
import pandas as pd
!pip install pulp brotli fuzzywuzzy
import pulp as plp
import sys 
import os
from collections import defaultdict
from fuzzywuzzy import process
import time


[notice] A new release of pip is available: 23.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
solve_season = '2024-25'
solve_gameweek = 27
load_projections_from_file = False

In [26]:
# Get the absolute path to the directory containing the Python file
module_path = os.path.abspath(os.path.join('..', '..'))

# Add the directory to sys.path
if module_path not in sys.path:
    sys.path.append(module_path)

# Now import the module
from projections import generate_projections, generate_stat_projections, append_stat_projections, calculate_manager_predictions

if not load_projections_from_file:
    point_projections = generate_projections()
    stat_projections = generate_stat_projections()

    projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)
    print('Generated new projections.')
else:
    try:
        projections_data = pd.read_csv('gameweek_projections.csv')
        print('Loaded projections from CSV file.')
    except FileNotFoundError:
        print('Tried to load from CSV file, but it does not exist. Generating new projections...')
        point_projections = generate_projections()
        stat_projections = generate_stat_projections()
        projections_data = append_stat_projections(point_projections, stat_projections, solve_gameweek)

Generated new projections.


### Player Manipulation

### Player Force/Banning

In [27]:
def fuzzy_ban_players(df, ban_ids):
    while True:
        search_name = input("Enter player name to ban (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to ban (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    ban_ids.append(selected_index)
                    print(f"Banned: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return ban_ids

ban_ids = []
ban_ids = fuzzy_ban_players(projections_data, ban_ids)
print("Final ban list (indices):", ban_ids)

Final ban list (indices): []


In [28]:
def fuzzy_force_players(df, force_ids):
    while True:
        search_name = input("Enter player name to force (or press enter to finish): ").strip()
        
        if search_name.lower() == '':
            break
        
        # Perform fuzzy matching with a lower score cutoff and no limit
        matches = process.extractBests(search_name, df['Name'].tolist(), score_cutoff=50, limit=10)
        
        if not matches:
            print("No matches found. Please try again.")
            continue
        
        # Display matches
        print("Matches found:")
        for idx, (name, score) in enumerate(matches, 1):
            player_index = df[df['Name'] == name].index[0]
            player_id = df.loc[player_index, 'ID']
            print(f"{idx}. {name} (ID: {player_id}, Index: {player_index}, Score: {score})")
        
        # Ask user to select a match
        while True:
            choice = input("Enter the number of the player to force (or 'skip' to search again): ")
            if choice.lower() == 'skip':
                break
            try:
                choice_idx = int(choice) - 1
                if 0 <= choice_idx < len(matches):
                    selected_name = matches[choice_idx][0]
                    selected_index = df[df['Name'] == selected_name].index[0]
                    selected_id = df.loc[selected_index, 'ID']
                    force_ids.append(selected_index)  # Add to force_ids instead of ban_ids
                    print(f"Forced: {selected_name} (ID: {selected_id}, Index: {selected_index})")
                    break
                else:
                    print("Invalid choice. Please try again.")
            except ValueError:
                print("Invalid input. Please enter a number or 'skip'.")
    
    return force_ids

force_ids = []
force_ids = fuzzy_force_players(projections_data, force_ids)
print("Final force list (indices):", force_ids)

Final force list (indices): []


# 2024/25 GW27 Challenge: Team Chemistry - Players from the Assistant Manager's club earn double points

In [29]:
from projections import calculate_manager_predictions
assistant_manager_points = calculate_manager_predictions(solve_gameweek=solve_gameweek)

display(assistant_manager_points)

,Team,Cost,Predicted_Points
0,Arsenal,1.5,6.75
1,Aston Villa,0.8,4.48
2,Brighton,1.1,5.32
3,Bournemouth,1.1,4.60
4,Brentford,0.8,5.87
5,Chelsea,1.5,8.76
6,Crystal Palace,0.8,10.77
7,Everton,0.5,3.84
8,Fulham,1.1,5.14
9,Ipswich,0.5,2.74


### Optimisation

In [32]:
import pulp as plp
import time
from collections import defaultdict

# Get the number of players and their list of ids
player_ids = projections_data['ID'].tolist()
player_count = len(player_ids)

# Set up the problem
model = plp.LpProblem("fpl-challenge", plp.LpMaximize)

# Define the decision variables
lineup = [plp.LpVariable(f"lineup_{i}", lowBound=0, upBound=1, cat="Integer") for i in player_ids]
captain = [plp.LpVariable(f"captain_{i}", lowBound=0, upBound=1, cat="Integer") for i in player_ids]
assistant_manager = [plp.LpVariable(f"am_{team}", lowBound=0, upBound=1, cat="Integer") for team in assistant_manager_points['Team']]

# Create mapping from team to assistant manager index
team_to_am_idx = {row['Team']: idx for idx, row in assistant_manager_points.iterrows()}

# Map each player to their assistant manager index
player_to_am_idx = [team_to_am_idx[projections_data.loc[i, 'Team']] for i in range(player_count)]

# Define variables for team matching
match_am_lineup = [plp.LpVariable(f"match_am_lineup_{i}", lowBound=0, upBound=1, cat="Integer") for i in range(player_count)]
match_am_captain = [plp.LpVariable(f"match_am_captain_{i}", lowBound=0, upBound=1, cat="Integer") for i in range(player_count)]

# Add constraints for team matching variables
for i in range(player_count):
    j_i = player_to_am_idx[i]
    # Make sure match_am_lineup is 1 only if both lineup and assistant manager are selected
    model += match_am_lineup[i] <= lineup[i]
    model += match_am_lineup[i] <= assistant_manager[j_i]
    model += match_am_lineup[i] >= lineup[i] + assistant_manager[j_i] - 1
    # Same for captain matching
    model += match_am_captain[i] <= captain[i]
    model += match_am_captain[i] <= assistant_manager[j_i]
    model += match_am_captain[i] >= captain[i] + assistant_manager[j_i] - 1

# Set the objective function (points from lineup, captain, team matching, and assistant manager)
model += (
    plp.lpSum([lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) +  # Base points
    plp.lpSum([match_am_lineup[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) +  # Extra points for team match
    plp.lpSum([captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) +  # Captain bonus
    plp.lpSum([match_am_captain[i] * projections_data.loc[i, 'Predicted_Points'] for i in range(player_count)]) +  # Extra captain points for team match
    plp.lpSum([assistant_manager[i] * assistant_manager_points.iloc[i]['Predicted_Points'] for i in range(len(assistant_manager_points))])  # Assistant manager points
)

# Constraints
model += plp.lpSum(lineup) == 10  # Total number of players = 10
for id in ban_ids:
    model += lineup[id] == 0  # Exclude banned players
for id in force_ids:
    model += lineup[id] == 1  # Force include specific players
model += plp.lpSum(captain) == 1  # Exactly one captain
model += plp.lpSum(assistant_manager) == 1  # Exactly one assistant manager
for i in range(player_count):
    model += captain[i] <= lineup[i]  # Captain must be in the lineup
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Goalkeeper']) == 1  # Exactly 1 Goalkeeper
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) >= 3  # 3-5 Defenders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Defender']) <= 5
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) >= 2  # 2-5 Midfielders
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Midfielder']) <= 5
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) >= 1  # 1-3 Forwards
model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Position'] == 'Forward']) <= 3
model += plp.lpSum([lineup[i] * projections_data.loc[i, 'Cost'] for i in range(player_count)]) + \
         plp.lpSum([assistant_manager[i] * assistant_manager_points.iloc[i]['Cost'] for i in range(len(assistant_manager_points))]) <= 100  # Budget of 100m
for team in assistant_manager_points['Team']:
    model += plp.lpSum([lineup[i] for i in range(player_count) if projections_data.loc[i, 'Team'] == team]) + \
             assistant_manager[team_to_am_idx[team]] <= 5  # Max 5 players per team including assistant manager

# Solve the problem
model.solve()

# Function to print players by position
def print_players_by_position(players_dict, selected_am):
    selected_am_team = selected_am['Team']
    total_points = 0
    total_cost = 0
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        if position in players_dict:
            print(f"\n{position}:")
            for player in players_dict[position]:
                base_points = player['Predicted_Points'] * (2 if player['Team'] == selected_am_team else 1)  # Double points if team matches
                points = base_points * (2 if player['Captain'] else 1)  # Double again if captain
                captain_str = " (C)" if player['Captain'] else ""
                print(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}")
                total_points += points
                total_cost += player['Cost']
    
    # Print assistant manager info
    am_points = selected_am['Predicted_Points']
    am_cost = selected_am['Cost']
    print(f"\nAssistant Manager:")
    print(f"  {selected_am['Team']} - Cost: {am_cost}m - Predicted Points: {am_points}")
    total_points += am_points
    total_cost += am_cost
    
    print(f"\nTotal Predicted Points: {round(total_points, 2)}")
    print(f"Total Cost: {round(total_cost, 2)}m")

# Function to save solution to text file
def solution_to_txt(players_dict, selected_am, filename="solution.txt", encoding="utf-8"):
    selected_am_team = selected_am['Team']
    total_points = 0
    total_cost = 0
    with open(filename, 'w', encoding=encoding) as f:
        f.write(f'Current Date & Time: {time.strftime("%Y-%m-%d")} - {time.strftime("%H:%M:%S")}\n')
        for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
            if position in players_dict:
                f.write(f"\n{position}:\n")
                for player in players_dict[position]:
                    base_points = player['Predicted_Points'] * (2 if player['Team'] == selected_am_team else 1)  # Double points if team matches
                    points = base_points * (2 if player['Captain'] else 1)  # Double again if captain
                    captain_str = " (C)" if player['Captain'] else ""
                    f.write(f"  {player['Name']}{captain_str} - {player['Team']} - Cost: {player['Cost']}m - Predicted Points: {points}\n")
                    total_points += points
                    total_cost += player['Cost']
        
        # Write assistant manager info
        am_points = selected_am['Predicted_Points']
        am_cost = selected_am['Cost']
        f.write(f"\nAssistant Manager:\n")
        f.write(f"  {selected_am['Team']} - Cost: {am_cost}m - Predicted Points: {am_points}\n")
        total_points += am_points
        total_cost += am_cost
        
        f.write(f"\nTotal Predicted Points: {round(total_points, 2)}\n")
        f.write(f"Total Cost: {round(total_cost, 2)}m\n")

# Print the results
print("Status:", plp.LpStatus[model.status])
selected_players = defaultdict(list)
for i in range(player_count):
    if lineup[i].value() == 1:
        player = projections_data.loc[i]
        selected_players[player['Position']].append({
            'Name': player['Name'],
            'Team': player['Team'],
            'Cost': player['Cost'],
            'Predicted_Points': player['Predicted_Points'],
            'Captain': captain[i].value() == 1
        })

# Get selected assistant manager
selected_am_idx = [i for i in range(len(assistant_manager)) if assistant_manager[i].value() == 1][0]
selected_am = assistant_manager_points.iloc[selected_am_idx]

# Save to text file
solution_to_txt(selected_players, selected_am, "optimal_solution.txt")

# Optionally print the solution
print_solution = False
if print_solution:
    print("\nOptimal Lineup:")
    print_players_by_position(selected_players, selected_am)

Status: Optimal

Optimal Lineup:

Goalkeeper:
  Jørgensen - Chelsea - Cost: 4.2m - Predicted Points: 8.28

Defender:
  Gabriel - Arsenal - Cost: 6.3m - Predicted Points: 4.67
  Cucurella - Chelsea - Cost: 5.1m - Predicted Points: 8.94
  Alexander-Arnold - Liverpool - Cost: 7.4m - Predicted Points: 5.04

Midfielder:
  Mbeumo - Brentford - Cost: 8.0m - Predicted Points: 5.88
  Nkunku - Chelsea - Cost: 5.7m - Predicted Points: 12.74
  Palmer (C) - Chelsea - Cost: 11.1m - Predicted Points: 35.12
  M.Salah - Liverpool - Cost: 13.7m - Predicted Points: 7.9
  Son - Spurs - Cost: 9.7m - Predicted Points: 5.93

Forward:
  Haaland - Man City - Cost: 14.7m - Predicted Points: 6.83

Assistant Manager:
  Chelsea - Cost: 1.5m - Predicted Points: 8.76

Total Predicted Points: 110.09
Total Cost: 87.4m


In [33]:
def print_top_scorers_by_position(projections_data):
    for position in ['Goalkeeper', 'Defender', 'Midfielder', 'Forward']:
        top_scorers = projections_data[projections_data['Position'] == position].nlargest(10, 'Predicted_Points')
        print(f'Top {position}s: ')
        display(top_scorers)

if print_solution:
    print_top_scorers_by_position(projections_data)

projections_data.to_csv('gameweek_projections.csv', index=False)

Top Goalkeepers: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
180,310,A.Becker,Liverpool,30,Goalkeeper,5.5,4.27,90,Newcastle (H),0.0,0.0,0.0,0.341,0.0
9,15,Raya,Arsenal,200,Goalkeeper,5.5,4.21,90,Nott'm Forest (A),0.0,0.0,0.0,0.421,0.0
342,581,Jørgensen,Chelsea,58,Goalkeeper,4.2,4.14,90,Southampton (H),0.0,0.0,0.0,0.444,0.0
57,91,Flekken,Brentford,152,Goalkeeper,4.4,4.09,90,Everton (H),0.0,0.0,0.0,0.364,0.0
228,383,Onana,Man Utd,38,Goalkeeper,5.0,3.94,90,Ipswich (H),0.0,0.0,0.0,0.444,0.0
325,554,José Sá,Wolves,173,Goalkeeper,4.3,3.90,90,Fulham (H),0.0,0.0,0.0,0.286,0.0
302,513,Areola,West Ham,73,Goalkeeper,4.2,3.79,90,Leicester (H),0.0,0.0,0.0,0.381,0.0
431,764,Palmer,Ipswich,241,Goalkeeper,4.5,3.77,90,Man Utd (A),0.0,0.0,0.0,0.154,0.0
143,248,Leno,Fulham,80,Goalkeeper,5.0,3.69,90,Wolves (A),0.0,0.0,0.0,0.308,0.0
133,235,Pickford,Everton,241,Goalkeeper,5.1,3.67,90,Brentford (A),0.0,0.0,0.0,0.231,0.0


Top Defenders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
181,311,Alexander-Arnold,Liverpool,241,Defender,7.4,5.04,84,Newcastle (H),0.076,0.265,0.321,0.341,0.087
1,3,Gabriel,Arsenal,30,Defender,6.3,4.67,90,Nott'm Forest (A),0.161,0.212,0.338,0.421,0.230
92,163,Cucurella,Chelsea,200,Defender,5.1,4.47,90,Southampton (H),0.066,0.022,0.086,0.444,0.077
117,211,Muñoz,Crystal Palace,48,Defender,4.9,4.38,90,Aston Villa (H),0.100,0.000,0.101,0.294,0.116
194,335,Robertson,Liverpool,243,Defender,5.9,4.31,78,Newcastle (H),0.050,0.034,0.082,0.341,0.060
91,162,Colwill,Chelsea,241,Defender,4.4,4.19,90,Southampton (H),0.084,0.000,0.084,0.444,0.096
317,533,Aït-Nouri,Wolves,3,Defender,4.7,4.19,81,Fulham (H),0.036,0.000,0.036,0.286,0.048
231,388,Wan-Bissaka,West Ham,241,Defender,4.4,4.14,90,Leicester (H),0.040,0.000,0.040,0.381,0.050
197,339,Virgil,Liverpool,152,Defender,6.4,4.10,90,Newcastle (H),0.104,0.107,0.200,0.364,0.117
53,86,Ajer,Brentford,161,Defender,4.4,4.05,90,Everton (H),0.034,0.047,0.079,0.359,0.041


Top Midfielders: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
103,182,Palmer,Chelsea,241,Midfielder,11.1,8.78,90,Southampton (H),0.580,0.062,0.606,0.444,0.902
191,328,M.Salah,Liverpool,63,Midfielder,13.7,7.90,90,Newcastle (H),0.472,0.564,0.770,0.364,0.675
102,181,Nkunku,Chelsea,73,Midfielder,5.7,6.37,79,Southampton (H),0.427,0.058,0.460,0.417,0.618
295,503,Son,Spurs,114,Midfielder,9.7,5.93,85,Man City (H),0.309,0.004,0.312,0.125,0.391
62,99,Mbeumo,Brentford,38,Midfielder,8.0,5.88,90,Everton (H),0.344,0.000,0.344,0.364,0.435
218,366,B.Fernandes,Man Utd,173,Midfielder,8.3,5.56,90,Ipswich (H),0.325,0.137,0.417,0.444,0.416
205,348,Foden,Man City,241,Midfielder,9.3,5.48,83,Spurs (A),0.320,0.005,0.324,0.234,0.424
46,74,O.Dango,Bournemouth,35,Midfielder,5.2,5.16,89,Brighton (A),0.250,0.000,0.250,0.200,0.299
303,514,Bowen,West Ham,241,Midfielder,7.3,5.14,88,Leicester (H),0.351,0.012,0.359,0.381,0.458
7,13,Ødegaard,Arsenal,161,Midfielder,8.2,4.91,89,Nott'm Forest (A),0.262,0.003,0.265,0.421,0.302


Top Forwards: 


,ID,Name,Team,Region,Position,Cost,Predicted_Points,xMins,Opponent,Score,Assist,Goal_Involvement,Clean_Sheet,Projected_Goals
207,351,Haaland,Man City,161,Forward,14.7,6.83,88,Spurs (A),0.547,0.018,0.555,0.250,0.856
240,401,Isak,Newcastle,206,Forward,9.5,5.83,87,Liverpool (A),0.282,0.082,0.341,0.104,0.348
322,541,Cunha,Wolves,30,Forward,6.9,5.65,90,Fulham (H),0.328,0.000,0.328,0.286,0.415
114,207,Mateta,Crystal Palace,73,Forward,7.4,5.54,90,Aston Villa (H),0.268,0.000,0.268,0.294,0.331
68,110,Wissa,Brentford,50,Forward,6.4,5.32,90,Everton (H),0.268,0.000,0.268,0.364,0.331
35,58,Watkins,Aston Villa,241,Forward,8.9,5.25,81,Crystal Palace (A),0.298,0.001,0.298,0.275,0.376
267,447,Wood,Nott'm Forest,154,Forward,7.2,5.02,88,Arsenal (H),0.228,0.000,0.228,0.257,0.277
146,252,Raúl,Fulham,139,Forward,5.6,4.64,80,Wolves (A),0.294,0.040,0.322,0.292,0.371
176,306,Vardy,Leicester,241,Forward,5.4,4.50,81,West Ham (A),0.241,0.000,0.241,0.167,0.303
331,566,Strand Larsen,Wolves,161,Forward,5.2,4.25,81,Fulham (H),0.000,0.000,0.000,0.000,0.000
